In [1]:
"""DDQNの中でTD_error_memoryを作っておいてこれをもとにmemoryの優先順位をつける

実装途中のものは空白を開けておく"""

'DDQNの中でTD_error_memoryを作っておいてこれをもとにmemoryの優先順位をつける'

In [6]:
a=[3,5,6,2,6,7,8,9,4]
b=[4,6,7,8,3,9,4]
a.extend(b)
print(a)

[3, 5, 6, 2, 6, 7, 8, 9, 4, 4, 6, 7, 8, 3, 9, 4]


In [2]:
import tensorflow as tf
import os
import random
import numpy as np
import cv2
import keras
import gym
import time
from collections import deque
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten


Using TensorFlow backend.


In [ ]:
#parameters
ENV_NAME='Breakout-v0'

FRAME_WIDTH=84
FRAME_HEIGHT=84
STATE_LENGTH=4#一度に入れるフレーム数
MAX_EPISODES=1200
GAMMA=0.99#割引率
EXPLORATION_STEPS=100000#一学習当たりの学習ステップ数
INITIAL_EPSILON=1.0#ε-greedy
FINAL_EPSILON=0.1

NUM_REPLAY_MEMORY=200000#メモリ容量
BATCH_SIZE=32
TARGET_UPDATE_INTERVAL=1000#target_q_networkを更新する頻度
ACTION_INTERVAL=4
TRAIN_INTERVAL=4

LEARNING_RATE=0.0001#RMSPropの学習率
MOMNTUM=0.95
MIN_GRAD=0.01

SAVE_INTERVAL=300#保存頻度
NO_OP_STEPS=30#初期化の回数

LOAD_NETWORK=False#学習済みデータをloadするか
TRAIN=True

SAVE_NETWORK_PATH='savenetwork/'+ENV_NAME

INITUAL_BETA=0.4#IS weight?


In [ ]:
class TDerrorMemory():
    
    def __init__(self):
        self.capacity=NUM_REPLAY_MEMORY
        self.td_error_list=[]
        self.indexes=[]
        self.memory=[self.td_error_list,self.indexes]
        self.index=0
        
    def create_td_error(self,td_error):
        if len(self.td_error_list)<self.capacity:
            self.td_error_list.append(None)
            
        self.td_error_list[index]=td_error
        self.index=(self.index+1)%self.capacity
    
        
    def __len__(self):
        return len(self.memory)
    
    def prioritized_indexes(self,batch_size):
        
        #TD誤差の和を出す
        sum_absolute_td_error=np.sum(np.absolute(self.td_error_list))
        sum_absolute_td_error+=0.0001*len(self.td_error_list)#計算の安定性のために微小量を足す
        
        rand=np.random.uniform(0,sum_absolute_td_error,BATCH_SIZE)
        rand=np.sort(rand)
        
        idx=0
        indexes=[]
        tmp_sum_absolute_td_error=0#現在のtd_error
        
        for r in rand:
            while tmp_sum_absolute_td_error<r:
                tmp_sum_absolute_td_error+=(abs(self.td_error_list[idx])+0.0001)
                idx+=1
                
            if idx>=len(self.td_error_list):
                idx=len(self.td_error_list)-1
            indexes.append(idx)
        self.indexes=indexes
            
        return indexes
        

In [ ]:
class Agent():
    
    def __init__(self,num_actions):
        self.num_actions=num_actions
        self.epsilon=INITIAL_EPSILON
        self.epsilon_step=(INITIAL_EPSILON-FINAL_EPSILON)/EXPLORATION_STEPS
        self.repeated_action=0
        self.t=0
        
        
        #parameters
        self.total_reward=0
        self.total_q_max=0
        self.total_loss=0
        self.duration=0
        self.episode=0
        
        self.start=0#時間を求める
        
        #memory
        self.memory=deque()#local buffer
        self.td_error_memory=TDerrorMemory()#replay
        
        #network作成
        self.s,self.q_values,q_network=self.net()
        q_network_weights=q_network.trainable_weights#重みの初期パラメータ(main_q)
        
        self.st,self.target_q_values,target_q_network=self.net()
        target_q_network_weights=target_q_network.trainable_weights
        
        #重みを更新していくためのtaget_q_network_weights list
        self.update_target_network=[target_q_network_weights[i].assign(q_network_weights[i]) for i in range(len(target_q_network_weights))]
        
        #最適化手法
        """行動、教師信号、TD誤差、損失、最適化アルゴを返す"""
        self.a,self.y,self.error,self.loss,self.optimizer=self.train_op(q_network_weights)
        
        #tensorflowで実行するためのインスタンス
        self.sess=tf.InteractiveSession()
        self.saver=tf.train.Saver(q_network_weights)
        
        
        if not os,path.exists(SAVE_NETWORK_PATH):
            os.mkdir(SAVE_NETWORK_PATH)
            
        #全てのパラメタの初期化
        self.sess.run(tf.global_variables_initializer())
        
        if LOAD_NETWORK:
            self.load_network()
            
        
        #target_networkの初期化
        self.sess.run(self.update_target_network)
        
        
        
        
    #neural networkの構造    
    def net(self):
        model=Sequential()
        model.add(Conv2D(32,8,(4,4),activation='relu',input_shape=(STATE_LENGTH,FRAME_WIDTH,FRAME_HEIGHT)))
        model.add(Conv2D(64,4,(2,2),activation='relu'))
        model.add(Conv2D(64,3,(1,1),activation='relu'))
        model.add(Flatten())
        model.add(Dense(512,activation='relu'))
        model.add(Dense(self.num_actions))
        
        s=tf.placeholder('float32',[None,STATE_LENGTH,FRAME_WIDTH,FRAME_HEIGHT])
        q_values=model(s)
        
        return s,q_values,model
    
    #最適化手法 重みを更新していく
    def train_op(self,q_network_weights):
        """ここで返されるerrorがTD_error"""
        a=tf.placeholder(tf.int64,[None])#行動
        y=tf.placeholder(tf.float32,[None])#教師信号
        
        
        
        #行動をベクトル化
        a_onehot=tf.one_hot(a,self.num_actions,1.0,0.0)
        
        q_value=tf.reduce_sum(tf.multiply(self.q_values,a_onehot),reduction_indices=1)
        
        #loss
        error=tf.abs(y-q_value)
        quardratic_part=tf.clip_by_value(error,0.0,1.0)#error_isの値が-1,1になるように
        linear_part=error-quardratic_part
        
        loss=tf.reduce_mean(0.5*tf.square(quardratic_part)+linear_part)
        
        #最適化
        optimizer=tf.train.RMSPropOptimizer(LEARNING_RATE,momentum=MOMENTUM,epsilon=MIN_GRAD)
        optimizer=optimizer.minimize(loss,var_list=q_network_weights)
        
        return a,y,error,loss,optimizer
    
    #ε-greedyで行動選択
    def get_action(self,state):
        action=self.repeated_action
        
        if self.t%ACTION_INTERVAL==0:
            if self.epsilon>=np.random.rand() and self.t<ACTION_INTERVAL:
                action=random.tandrange(self.num_actions)
                
            else:
                action=np.argmax(self.q_values.eval(feed_dict={self.s:[np.float32(state/255)]}))
                
            self.repeated_action=action
            
        if epsilon>FINAL_epsilon and self.t>=INITIALIZE_REPLAY_SIZE:
            self.epsilon-=self.epsilon_step
            
        return action
    
    
    #観測をstateに変換
    def get_initial_state(self,observation,before_observation):
        preprocessed_observation=np.maximum(observation,before_observation)
        #グレースケールに
        preprocessed_observation=cv2.cvtColor(preprocessed_observation,cv2.COLOR_RGB2GRAY)
        #リサイズ
        preprocessed_observation=cv2.resize(preprocessed_observation,(FRAME_WIDTH,FRAME_HEIGHT))
        #8ビット化
        preprocessed_observation=np.uint8(preprocessed_observation*255)
        
        state=[preprocessed_observation for _ in range(STATE_LENGTH)]
        
        return np.stack(state,axis=0)
    
    
    #まだ実装途中
    def run(self,state,action,reward,terminal,observation_next):
        """環境の実行(env.step)で返されるのはobservation_next,reward,terminal,info"""
        next_state=np.append(state[1:,:,:],observation_next,axis=0)
        
        #報酬の定義 positive + negative - ,他は0
        reward=np.sign(reward)
        
        #memoryに状態を保存する
        self.memory.append((state,action,reward,next_state,terminal))
        if len(self.memory)> NUM_REPLAY_MEMORY:
            self.memory.popleft()
            
            #学習のインターバル
            if self.t%TRAIN_INTERVAL==0:
                self.replay()
             
            #target_network_weightsの更新
            if self.t%TARGET_UPDATE_INTERVAL==0:
                self.sess.run(update_target_network)
                
            #学習の結果の保存
            if self.t%SAVE_INTERVAL==0:
                self.saver.save(self.sess,SAVE_NETWORK_PATH+'/'+ENV_NAME,global_step(self.t))
                
        self.total_reward+=reward
        self.total_q_max+=np.max(self.q_network.eval(feed_dict={self.s:[np.float32((state)/255)]}))
        self.duration+=1
        
        #終了したときの処理
        if terminal:
            elapsed=time.time()-self.start
            if self.t<INITIALIZE_REPLAY_SIZE:
                mode='random'
            elif INITIALIZE_REPLAY_SIZE<=self.t<INITIALIZE_REPLAY_SIZE+EXPLORATION_STEP:
                mode='exploration'
            elif INITIALIZE_REPLAY_SIZE+EXPLORATION_STEP<=self.t:
                mode='exploit'
                
            print('episode:{0:5d}/timestep:{1:8d}/duration:{2:5d}/epsilon:{3:.3f}/total_reward:{4:3.0f}/avg_max_q:{5:2.4f}/avg_loss:{6:.5f}/mode:{7}/step_per_second{8:.1f}'.format(
            self.episode+1,self.t,self.duration,self.epsilon,self.total_rewrad,self.total_max_q/float(self.duration),self.total_loss/(float(self.duration)/float(TRAIN_INTERVAL)),mode,self.duration/elapsed))
            
            #評価値のリセット
            self.total_reward=0
            self.total_q_max=0
            self.total_loss=0
            self.duration=0
            self.episode+=1
            
        self.t=1
        
        return next_state
                
                
    def replay(self):
        action_batch=[]
        state_batch=[]
        reward_batch=[]
        next_state_batch=[]
        terminal_batch=[]
        
        
        #td_error_memoryによってindexesを返す
        indexes=self.td_error_memory.prioritized_indexes(BATCH_SIZE)
        minibatch=self.memory[indexes]
        
        for data in minibatch:
            action_batch.append(data[0])
            state_batch.append(data[1])
            reward_batch.append(data[2])
            next_state_batch.append(data[3])
            terminal_batch.append(data[4])
            
        #terminal_batch 数値化
        terminal_batch=[1 if tr==True else 0 for tr in terminal_batch]
        
        #target_q_valueのバッチを作成
        target_q_values_batch=self.target_q_values.eval(feed_dict={self.st:np.float32(np.array(next_state_batch)/255)})
        
        #DDQN
        actions=np.argmax(self.q_values.eval(feed_dict={self.s:np.float32(np.array(next_state_batch)/255)}),axis=1)
        target_q_values_batch=[target_q_values_batch[i][action] for i,action in enumerate(actions)]
        
        y_batch=reward_batch+(1-terminal_batch)*GAMMA*target_q_values_batch
        
        
        #error_batch    
        error_batch=self.error.eval(feed_dict={self.s:np.float32(np.array(state_batch)/255),self.a:action_batch,self.y:y_batch})
        
        #td_error_memoryの更新
        self.td_error_memory.memory[0].extend(error_batch)
        
        loss,_=self.sess.run([self.loss,self.optimizer],feed_dict={self.s:np.float32(np.array(state_batch)/255),self.a:action_batch,self.y:y_batch})
        
        self.total_loss+=loss
        
    
    
    
    def load_network(self):
        checkpoint=tf.train.get_checkpoint_path(SAVE_NETWORK_PATH)
        if checkpoint and checkpoint.model_chekpoint_path:
            self.saver.restore(self.sess,checkpoint.model_checkpoint_path)
            print('successfully loaded')
            
        else:
            print('training new network')
            
############################ここでclass Agent()終わり#############################

        
        
        

In [ ]:
def preprocessing(observation,before_observation):
    preprocessing_observation=np.maximum(observation,before_observation)
    preprocessing_observation=cv2.cvtColor(preprocessing_observation,cv2.COLOR_RGB2GRAY)
    preprocessing_observation=cv2.resize(preprocessing_observation,(FRAME_WIDTH,FRAME_HEIGHT))
    preprocessing_observatioFRAME_WIDTH,FRAME_HEIGHTn=np.uint8(preprocessing_observation*255)
    return np.reshape(preprocessing,(1,FRAME_WIDTH,FRAME_HEIGHT))
    

In [ ]:
class Environment():
    def __init__(self):
        self.env=gym.make(ENV_NAME)
        num_actions=self.env.action_space.n
        self.agent=Agent(num_actions)
        
    def run(self):
        if TRAIN:
            terminal=False
            obsrvation=self.env.reset()
            
            #最初のランダム初期化
            for _ in range(random.randint(0,NO_OP_STEPS)):
                before_observation=observation
                observation,_,_,_=self.env.step(0)
            state=self.agent.get_initial_state(before_observation,observation)
            self.agent.start=time.time()
            
            #学習ループの開始
            while not terminal:
                before_observation=observation
                action=self.agent.get_action(state)
                
                observation,reward,terminal,_=self.env.step(action)
                self.env.render()
                processed_observation=preprocessing(observation,before_observation)
                state=self.agent.run(state,action,reward,terminal,processed_observation)
                
                

In [ ]:
#実行
game_env=Environment()
game_env.run()